# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [ ]:
import xlwings as xw
import numpy as np
import pandas as pd

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [ ]:
wb = xw.Book('data\\себестоимостьА_в1.xlsx') 
sht = wb.sheets['Рецептура']
val = pd.DataFrame(sht.range("G7:O10").value)
price = sht.range("G14:O14").value
cost_price = (val*price).sum(axis=1)
wb.save()

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [ ]:
new = cost_price.apply(lambda x: [x]).values
sht.range('T7').options(np.array, expand='table', ndim=1).value = new
wb.save()

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
print(sht.range('P7:P10').color)
sht.range('T7:T10').color = sht.range('P7:P10').color
sht.range('T7:T10').api.Borders.Weight = sht.range('P7:P10').api.Borders.Weight
wb.save()

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
import pandas as pd

s = ['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']
reviews = pd.read_csv('reviews_sample.csv')
recipes = pd.read_csv('recipes_sample.csv')
#print(reviews)
#print(recipes)
for i in recipes:
    if i not in s:
        recipes.drop(columns = [i], axis = 1, inplace = True)

reviews


,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
import openpyxl
import numpy as np
import pandas as pd
import xlsxwriter

writer = pd.ExcelWriter('recipes.xlsx')

recipes[:150].to_excel(writer, 'Рецепты')
reviews[:6334].to_excel(writer, 'Отзывы')


3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
seconds = pd.read_excel('data\\recipes.xlsx', sheet_name="Рецепты").minutes*60

wb3 = xw.Book('data\\recipes.xlsx')
recipes_sheet = wb3.sheets['Рецепты']

new = seconds.apply(lambda x: [x]).values
recipes_sheet.range("J1").value=["seconds_assign"]
recipes_sheet.range('J1').font.bold = True

recipes_sheet.range('J2').options(np.array, expand='table', ndim=1).value = new


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
wb3 = xw.Book('data\\recipes.xlsx')
recipes_sheet = wb3.sheets['Рецепты']

range_name="D"
first_row_index = recipes_sheet.range("D1").end('left').row
last_row_index = recipes_sheet.range("D1").end('down').row
row_count = last_row_index - first_row_index
recipes_sheet.range("K1").value=["seconds_formula"]
recipes_sheet.range('K1').font.bold = True

for i in range(2, row_count + 2):
    recipes_sheet.range(f"K{i}").value = [f"={range_name}{i}*60"]
wb3.save()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
wb5 = xw.Book('data\\recipes.xlsx')
recipes_sheet = wb5.sheets['Рецепты']
recipes_sheet.range("J1:K1").api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
recipes_sheet.range("J1:K1").font.bold = True
wb5.save()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
wb6 = xw.Book('data\\recipes.xlsx')
recipes_sheet = wb6.sheets['Рецепты']
row_count = 1500
column_name = "D"
for i in range(2, row_count+2):
    val = recipes_sheet.range(f"D{i}").value
    print(val)
    if val < 5:
        recipes_sheet.range(f"D{i}").color = (124, 252, 0)
        print(5)
    elif 5 <= val <= 10:
        recipes_sheet.range(f"D{i}").color = (255, 255, 167)
        print(510)
    elif val > 10:
        recipes_sheet.range(f"D{i}").color = (249, 138, 138)
        print(10)
wb6.save()
wb6.close()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
wb7 = xw.Book('data\\recipes.xlsx')
recipes_sheet = wb7.sheets['Рецепты']
recipes_sheet.range("L1").value = "n_reviews"
recipes_sheet.range("L1").api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
recipes_sheet.range("L1").font.bold = True
recipes_sheet.range(f'L2:L{row_count}').formula = f'=COUNTIF(Отзывы!D:D, B2)'
wb7.save()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
import xlwings as xw
def validate(recipes_sheet, reviews_sheet, row_count = 1500):
    reviews_sheet.range(f'L2:L{row_count}').formula = f'=COUNTIF(Рецепты!B:B, D2)'
    for i in range(2, row_count+2):
        rating_val = reviews_sheet.range(f"F{i}").value
        id_val = recipes_sheet.range(f"B{i}").value
        recipe_id_val = reviews_sheet.range(f"D{i}").value
        rec_in = reviews_sheet.range(f"L{i}").value
        if not(0<=rating_val<=5 and rec_in):
            reviews_sheet.range(f"A{i}:L{i}").color = (249, 138, 138)
            
wb6 = xw.Book('data\\recipes.xlsx')
recipes_sheet = wb6.sheets['Рецепты']
reviews_sheet = wb6.sheets['Отзывы']

first_row_index = reviews_sheet.range("D1").end('left').row
last_row_index = reviews_sheet.range("D1").end('down').row
row_count = last_row_index - first_row_index
column_name = "D"
validate(recipes_sheet, reviews_sheet, row_count=row_count)

wb6.save()
wb6.close()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
model = pd.read_csv("data\\recipes_model.csv", delimiter="\t", header=None)
model.to_excel("data\\recipes_model.xlsx", sheet_name="Модель", startrow=1, index=False, header=False)

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
wb11 = xw.Book("data\\recipes_model.xlsx")
sht11 = wb11.sheets["Модель"]


sht11.range("G1").value = "Ключ"
sht11.range("J1").value = "SQL скрипт создания строки"

sht11.range(f"A1:J1").color = "#00ccff"

sht11.autofit(axis="columns")

sht11.range("A1:J1").font.bold = True

sht11.show_autofilter = True
wb11.save()

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 